# **🏠 부동산 실거래가 Baseline code**
> 부동산 실거래가 예측 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 RandomForest를 활용해 ML 방법론들을 실제 대회에 적용해보도록 하겠습니다.
> 강의는 google colab으로 실행하였기에 아래의 베이스라인 코드와는 일부 차이가 있을 수 있습니다. 

## Contents
- Library Import
- Data Load
- Data Preprocessing
- Feature Engineering
- Model Training
- Inference
- Output File Save

## 프로젝트 후기: 부동산 거래가 예측 EDA

이번 EDA 과정은 단순한 데이터 탐색을 넘어서, 실제 모델 성능에 기여할 수 있는 인사이트를 추출하는 데 중점을 두었습니다. 특히 다음과 같은 점이 인상 깊었습니다:

---

### 인상 깊었던 점

- **도메인 지식 활용**:  
  강남권 여부, 계약년월, 구/동 분리, 따릉이 위치정보, 지하철역 위치정보 등의 파생변수를 통해 지역성과 시간성을 반영할 수 있었고, 이는 target 예측과의 연관성이 뚜렷하게 나타났습니다.

- **거리 기반 파생변수**:  
  지하철·따릉이와의 거리 및 접근성 변수를 추가하면서 위치 데이터가 가진 가치를 수치로 표현할 수 있었습니다.

- **결측치 분석의 정량화**:  
  단순히 결측 비율이 높은 변수만 제거하지 않고, target과의 상관관계를 함께 고려한 합리적 판단을 내릴 수 있었습니다.

---

### 아쉬웠던 점

- 로그 변환/클리핑 기준을 더 체계적으로 정량화하지 못한 점
- 변수 간 다중공선성 처리 및 변수 선택에 대한 자동화 기법은 향후 보완할 필요가 있음

---

### 다음 목표

- **LightGBM 모델 적용**:  
  이번 EDA에서 정리된 변수를 바탕으로 모델링을 수행하고 성능 평가

- **파생 변수 추가 생성**:
  강력한 파생 변수 생성이 주요한 성능 향상에 크게 기여할 것임을 예측
  베이스라인 데이터셋의 한계가 보임

- **Feature Importance 분석**:  
  어떤 피처들이 실제로 모델 성능에 기여하는지 확인

- **Shap, Permutation Importance 등 해석 기법 적용**


```python
# 데이터 로딩
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
```
> `train.csv`, `test.csv` 불러와 `is_test`로 합침



```python
# 테스트셋 구분 컬럼 추가
train['is_test'] = 0
test['is_test'] = 1
```

```python
# 전처리를 위한 통합 데이터 생성
concat = pd.concat([train, test], axis=0)
```

```python
# 구조 확인
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Concat shape:", concat.shape)
```
> train/test/concat의 shape 확인
Train shape: (1118822, 53)
Test shape: (9272, 52)
Concat shape: (1128094, 53)



```python
# 컬럼 정보 확인
print("\n[train.info()]")
train.info()
print("\n[test.info()]")
test.info()
```
> `.info()`로 컬럼별 데이터 타입, 결측 파악



```python
# 데이터 샘플 확인
display(train.head(2))
display(test.head(2))
```
```python
# 수치형 변수 통계 요약
display(train.describe().T)
```
```python
# 결측치 개수 및 비율 계산
missing = concat.isnull().sum()
missing_ratio = (missing / len(concat)).sort_values(ascending=False)
missing
```
> 전체 결측치 개수 및 비율 계산




```python
# 시각화 (상위 30개)
plt.figure(figsize=(14, 4))
missing_ratio.head(30).plot.bar(color='tomato')
plt.title('결측치 비율 (상위 30개)')
plt.ylabel('결측 비율')
plt.grid(True)
plt.tight_layout()
plt.show()
```
```python
# 100만개 이상 결측된 컬럼 목록
threshold = 1_000_000
cols_to_drop = missing[missing >= threshold].index.tolist()
cols_to_drop
```
```python
# 제거
concat = concat.drop(columns=cols_to_drop)
```
> 결측치 100만 이상 컬럼 제거




```python
# 남은 변수 분리
continuous_cols = []
categorical_cols = []

for col in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[col]):
        continuous_cols.append(col)
    else:
        categorical_cols.append(col)
```
> 수치형과 범주형 컬럼 구분




```python
# 특수처리: 숫자지만 범주형인 '본번', '부번'
for col in ['본번', '부번']:
    if col in continuous_cols:
        concat[col] = concat[col].astype(str)
        continuous_cols.remove(col)
        categorical_cols.append(col)
```
```python
# 결측치 채움
concat[categorical_cols] = concat[categorical_cols].fillna('NULL')
concat[continuous_cols] = concat[continuous_cols].interpolate(method='linear', axis=0)
```
```python
# 확인
print("결측치 남은 수 (상위):")
display(concat.isnull().sum().sort_values(ascending=False))
```
```python
# 결측치는 아닌데 의미 없는 형식적 값 찾기
def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if concat[col].dtype == 'object':
            val_counts = concat[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)
for col, vals in fake_nulls.items():
    print(f"* {col} 컬럼에서 의미 없는 값 발견:")
    print(vals)
    print()
```
> '-', '', '없음' 등 의미 없는 값 탐색




```python
# 의미 없는 값 정의
fake_null_values = ['-', ' ', '', '.', '없음', 'nan']
```
```python
# 문자형 컬럼 기준 일괄 변환
object_cols = concat.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    concat[col] = concat[col].replace(fake_null_values, np.nan)
```
```python
# 복사본 생성
concat_select = concat.copy()
```

```python
# 본번, 부번 → 문자형 변환
concat_select['본번'] = concat_select['본번'].astype(str)
concat_select['부번'] = concat_select['부번'].astype(str)
```

```python
# 연속형 변수 추출
continuous_cols = concat_select.select_dtypes(include=['int64', 'float64']).columns.tolist()
continuous_cols = [col for col in continuous_cols if col != 'is_test']  # is_test 제외
```

```python
# 범주형 변수 추출
categorical_cols = concat_select.select_dtypes(include=['object']).columns.tolist()
```

```python
# 결과 확인
print(f"연속형 변수 개수: {len(continuous_cols)}")
print(f"범주형 변수 개수: {len(categorical_cols)}")
print("연속형 변수:", continuous_cols)
print("범주형 변수:", categorical_cols)
```

```python
# 현재 수치형 변수 목록
train_only = concat[concat['is_test'] == 0].copy()
numeric_cols = train_only.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['is_test']]
```

```python
# 결측치 비율 계산
missing_ratio = train_only[numeric_cols].isnull().mean()
```

```python
# 상관계수 계산
corr_matrix = train_only[numeric_cols].corr()
target_corr = corr_matrix['target'].drop('target')
```
> 수치형 변수 간 상관계수 및 타겟과의 관계 분석




```python
# 결측비율 + 상관계수 조합 테이블
value_check = pd.DataFrame({
    '결측비율': missing_ratio,
    'target_상관계수': target_corr,
    '절대_상관': target_corr.abs()
}).sort_values(by='절대_상관', ascending=False)
```

```python
# 출력
```

```python
# 절대_상관 < 0.05 AND 결측비율 > 0.7 → 제거 강력 고려
```
> 결측치 100만 이상 컬럼 제거




```python
# 절대_상관 < 0.1 AND 결측비율 > 0.3 → 제거 또는 비지도화 후 사용 고려
display(value_check)
```
> 결측치 100만 이상 컬럼 제거




```python
# 수치형 변수 필터링
train_only = concat[concat['is_test'] == 0].copy()
numeric_cols = train_only.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['is_test']]
```

```python
# 상관계수 계산
corr = train_only[numeric_cols].corr()
```
> 수치형 변수 간 상관계수 및 타겟과의 관계 분석




```python
# 히트맵 시각화
plt.figure(figsize=(14, 12))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', center=0, square=True, linewidths=.5)
plt.title('수치형 변수 간 상관관계 히트맵')
plt.show()
```

```python
## 전용면적 이상치 클리핑 셀
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 복사본 생성
concat['전용면적_clip'] = concat['전용면적(㎡)'].copy()
```

```python
# 상하위 0.5% 경계 계산
q_low = concat['전용면적_clip'].quantile(0.005)
q_high = concat['전용면적_clip'].quantile(0.995)
```

```python
# 클리핑 적용
concat['전용면적_clip'] = concat['전용면적_clip'].clip(lower=q_low, upper=q_high)
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 결과 시각화
plt.figure(figsize=(10, 4))
sns.histplot(concat['전용면적_clip'], bins=100, kde=True, color='cornflowerblue')
plt.title('전용면적(㎡) 클리핑 후 분포')
plt.xlabel('전용면적(㎡)')
plt.ylabel('건수')
plt.grid(True)
plt.show()
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 경계 출력
print(f"[클리핑 경계]")
print(f"  하위 0.5%: {q_low:.2f}㎡")
print(f"  상위 99.5%: {q_high:.2f}㎡")
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
## k-주거전용면적과 k-전체세대수 사이에 상관관계가 있는지 확인
```

```python
# # 세대당 전용면적 파생 변수 생성
concat['세대당_전용면적'] = concat['k-주거전용면적'] / concat['k-전체세대수']
```

```python
# 요약 통계 확인
print("[기초 통계]")
print(concat['세대당_전용면적'].describe())
```

```python
# target과의 상관계수 확인
corr = concat[['세대당_전용면적', 'target']].corr().iloc[0, 1]
print(f"\n[target과의 상관계수]: {corr:.4f}")
```

```python
# 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 4))
sns.histplot(concat['세대당_전용면적'].dropna(), bins=80, kde=True, color='teal')
plt.title('세대당 전용면적 분포')
plt.xlabel('㎡')
plt.grid(True)
plt.show()
```

```python
## lgbm을 사용할 거기 때문에 로그스케일대신 클리핑을 채택
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 하위/상위 0.5% 경계 계산
low = concat['세대당_전용면적'].quantile(0.005)
high = concat['세대당_전용면적'].quantile(0.995)
```

```python
# 클리핑 적용
concat['세대당_전용면적_clip'] = concat['세대당_전용면적'].clip(lower=low, upper=high)
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 결과 확인
print(f"[클리핑 경계]")
print(f"  하위 0.5%: {low:.2f}㎡")
print(f"  상위 99.5%: {high:.2f}㎡")
```

```python
# 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 4))
sns.histplot(concat['세대당_전용면적_clip'].dropna(), bins=80, kde=True, color='slateblue')
plt.title('세대당 전용면적 (클리핑 후) 분포')
plt.xlabel('세대당 전용면적(㎡)')
plt.ylabel('Count')
plt.grid(True)
plt.show()


print("[기초 통계]")
print(concat['주차대수'].describe())
```

```python
# 주차대수 결측치 비율 및 target과의 상관계수 확인
print(f"[결측치 비율]: {concat['주차대수'].isna().mean():.4f}")
print(f"[target과의 상관계수]: {concat[['주차대수', 'target']].corr().iloc[0,1]:.4f}")
```

```python
# 주차대수 분포 시각화
plt.figure(figsize=(10, 4))
sns.histplot(concat['주차대수'].dropna(), bins=80, kde=True, color='teal')
plt.title('주차대수 분포')
plt.xlabel('주차대수')
plt.ylabel('Count')
plt.grid(True)
plt.show()
```

```python
## 각별한 이상치가 없으므로 로그 변환 채택
```
> 로그 스케일로 분포 정규화




```python
# 로그 변환
concat['주차대수_log'] = np.log1p(concat['주차대수'])
```
> 로그 스케일로 분포 정규화




```python
# 시각화
plt.figure(figsize=(10, 4))
sns.histplot(concat['주차대수_log'], bins=80, kde=True, color='mediumpurple')
plt.title('주차대수 (로그 변환 후) 분포')
plt.xlabel('log(주차대수 + 1)')
plt.ylabel('Count')
plt.grid(True)
plt.show()
```

```python
# 상관계수 확인
corr = concat[['주차대수_log', 'target']].corr().iloc[0,1]
print(f"[target과의 상관계수]: {corr:.4f}")
```

```python
# 연면적 분포 확인
plt.figure(figsize=(10, 5))
sns.histplot(concat['k-연면적'], bins=100, kde=True)
plt.title('연면적 분포')
plt.xlabel('k-연면적 (㎡)')
plt.ylabel('Count')
plt.show()
```

```python
# 기초 통계
print('[기초 통계]')
print(concat['k-연면적'].describe())
```

```python
# 결측치 및 상관계수
print('[결측치 비율]:', concat['k-연면적'].isna().mean())
print('[target과의 상관계수]:', concat[['k-연면적', 'target']].corr().iloc[0, 1])
```

```python
## 로그변환 먼저 해보고 클리핑할지 판단해보기로함
concat['k-연면적_log'] = np.log1p(concat['k-연면적'])
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 시각화
plt.figure(figsize=(10, 5))
sns.histplot(concat['k-연면적_log'], bins=100, kde=True, color='slateblue')
plt.title('연면적 (로그 변환 후) 분포')
plt.xlabel('log(k-연면적 + 1)')
plt.ylabel('Count')
plt.show()
```

```python
# 상관계수 확인
corr_val = concat[['k-연면적_log', 'target']].corr().iloc[0, 1]
print('[target과의 상관계수]:', round(corr_val, 4))
```

```python
## 분포가 이쁘게 나오긴했는데, 상관계수는 떨어짐. 클리핑도 해보고 결정하기로함
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 경계 설정
lower = concat['k-연면적'].quantile(0.005)
upper = concat['k-연면적'].quantile(0.995)
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 적용
concat['k-연면적_clipped'] = concat['k-연면적'].clip(lower=lower, upper=upper)
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 분포 시각화
plt.figure(figsize=(12, 5))
sns.histplot(concat['k-연면적_clipped'], bins=80, kde=True, color='skyblue')
plt.title('연면적 (클리핑 후) 분포')
plt.xlabel('k-연면적 (㎡)')
plt.ylabel('Count')
plt.tight_layout()
plt.grid(True)
plt.show()
```

```python
# 상관계수 계산
correlation = concat[['k-연면적_clipped', 'target']].corr().iloc[0, 1]
print(f'[클리핑 경계]: {lower:.2f}㎡ ~ {upper:.2f}㎡')
print(f'[target과의 상관계수]: {correlation:.4f}')
```
> 수치형 변수 간 상관계수 및 타겟과의 관계 분석




```python
## 클리핑 분포도 이쁜편이고, 상관계수도 훨씬 높기때문에, 클리핑으로 채택
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 기본 통계 및 결측/상관 정보
col = 'k-주거전용면적'
print("[기초 통계]")
print(concat[col].describe())
print(f"[결측치 비율]: {concat[col].isna().mean():.4f}")
print(f"[target과의 상관계수]: {concat['target'].corr(concat[col]):.4f}")
```

```python
# 분포 시각화
plt.figure(figsize=(10,5))
sns.histplot(concat[col], bins=100, kde=True)
plt.title(f'{col} 분포')
plt.xlabel(f'{col} (㎡)')
plt.ylabel('Count')
plt.show()
```

```python
## 분포가 이쁘진 않지만 로그 변환으로 해석을 어렵게 할 필요는 없어보이므로 이것도 클리핑
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 경계 계산 (0.5% ~ 99.5%)
low, high = np.percentile(concat['k-주거전용면적'], [0.5, 99.5])
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 클리핑 적용
concat['k-주거전용면적_clipped'] = concat['k-주거전용면적'].clip(lower=low, upper=high)
```
> 극단값 클리핑 (0.5% ~ 99.5%)




```python
# 분포 시각화
plt.figure(figsize=(10, 5))
sns.histplot(concat['k-주거전용면적_clipped'], bins=60, kde=True, color='skyblue')
plt.title('k-주거전용면적 (클리핑 후) 분포')
plt.xlabel('k-주거전용면적 (㎡)')
plt.ylabel('Count')
plt.grid(True)
plt.show()
```

```python
# 상관계수 확인
corr = concat[['k-주거전용면적_clipped', 'target']].corr().iloc[0, 1]
print(f"[클리핑 경계]: {low:,.2f}㎡ ~ {high:,.2f}㎡")
print(f"[target과의 상관계수]: {corr:.4f}")
```

```python
# concat에서 만든 파생변수들을 concat_select에 모두 복사
concat_select['전용면적_clip'] = concat['전용면적_clip']
concat_select['세대당_전용면적'] = concat['세대당_전용면적']
concat_select['세대당_전용면적_clip'] = concat['세대당_전용면적_clip']
concat_select['주차대수_log'] = concat['주차대수_log']
concat_select['k-연면적_log'] = concat['k-연면적_log']
concat_select['k-연면적_clipped'] = concat['k-연면적_clipped']
concat_select['k-주거전용면적_clipped'] = concat['k-주거전용면적_clipped']

concat_select.head(1)

concat_select.info()
```

```python
# 결측치 있는 컬럼만 전체 출력
null_summary = concat_select.isnull().sum()
null_summary = null_summary[null_summary > 0].sort_values(ascending=False)
```

```python
# 깔끔하게 출력
print(f"결측치가 있는 변수 수: {len(null_summary)}개")
display(null_summary)

drop_cols = ['등기신청일자', '중개사소재지', '거래유형', '도로명', 'k-시행사']
concat_select = concat_select.drop(columns=drop_cols)

print(f"Drop 완료: {len(drop_cols)}개 컬럼 제거")
```

```python
# 컬럼 동기화 (드롭된 컬럼 제거)
categorical_cols = [col for col in categorical_cols if col in concat_select.columns]
continuous_cols = [col for col in continuous_cols if col in concat_select.columns]
```

```python
# 연속형 결측치 존재 여부
cont_missing_count = concat_select[continuous_cols].isnull().sum()
cont_missing_count = cont_missing_count[cont_missing_count > 0]
```

```python
# 범주형 결측치 존재 여부
cat_missing_count = concat_select[categorical_cols].isnull().sum()
cat_missing_count = cat_missing_count[cat_missing_count > 0]
```

```python
# 결과 출력
print(f"결측치가 있는 연속형 변수 수: {len(cont_missing_count)}")
if not cont_missing_count.empty:
    display(cont_missing_count)

print(f"\n결측치가 있는 범주형 변수 수: {len(cat_missing_count)}")
if not cat_missing_count.empty:
    display(cat_missing_count)
```

```python
# 동으로 묶어서 파생변수를 만들려고보니 변수가 339개 생겨서 과적합의 이유로 포기
unique_시군구 = concat_select['시군구'].unique()
pd.DataFrame({'시군구': sorted(unique_시군구)}).reset_index(drop=True)
```

```python
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
concat_select['구'] = concat_select['시군구'].map(lambda x : x.split()[1])
concat_select['동'] = concat_select['시군구'].map(lambda x : x.split()[2])
del concat_select['시군구']

concat_select['계약년'] = concat_select['계약년월'].astype('str').map(lambda x : x[:4])
concat_select['계약월'] = concat_select['계약년월'].astype('str').map(lambda x : x[4:])
del concat_select['계약년월']

concat_select.columns

all = list(concat_select['구'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.
```

```python
# 강남의 여부를 체크합니다.
is_gangnam = []
for x in concat_select['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)
```
> 도메인 지식으로 강남구 여부 변수 생성




```python
# 파생변수를 하나 만릅니다.
concat_select['강남여부'] = is_gangnam
```

```python
# 확인
concat_select.columns
```

```python
# 전체 컬럼 수 확인
print(f"현재 변수 개수: {concat_select.shape[1]}개")
```

```python
# 필요 시 컬럼명도 같이 확인
print("\n변수 리스트:")
for i, col in enumerate(concat_select.columns):
    print(f"[{i+1}] {col}")

pd.DataFrame({'Index': range(len(concat_select.columns)), 'Column': concat_select.columns.tolist()})
```

```python
## 베이스코드 feature importance와 내 도메인 지식을 기반으로 제거할 컬럼을 정해봄
drop_indices = [8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 21, 26, 27, 28, 29, 32, 33, 34, 35, 38]
drop_cols = [concat_select.columns[i] for i in drop_indices]
print("드랍할 컬럼:")
print(drop_cols)

concat_select.drop(columns=drop_cols, inplace=True)
```
> 불필요한 컬럼 제거




```python
# 전체 컬럼 수 확인
print(f"현재 변수 개수: {concat_select.shape[1]}개")
```

```python
# 필요 시 컬럼명도 같이 확인
print("\n변수 리스트:")
for i, col in enumerate(concat_select.columns):
    print(f"[{i+1}] {col}")
```

```python
# 외부 파일 불러오기
subway = pd.read_csv('/data/ephemeral/home/Bkan/Competition_dataset/subway_feature.csv')  # 지하철역 위경도 데이터
bike = pd.read_csv('/data/ephemeral/home/Bkan/Competition_dataset/bike_station.csv', encoding='cp949')      # 따릉이 대여소 위경도 데이터
```

```python
# 샘플 및 컬럼 확인
print("Subway data sample:")
display(subway.head(), subway.columns)

print("\nBike data sample:")
display(bike.head(), bike.columns)
```

```python
# 아파트 기준 좌표 (위도, 경도)
apt_coords = np.vstack([concat_select['좌표Y'], concat_select['좌표X']]).T
```

```python
# 지하철 좌표
subway_coords = np.vstack([subway['위도'], subway['경도']]).T
subway_tree = cKDTree(subway_coords)
```

```python
# 따릉이 좌표 (결측/0.0 제거)
bike_clean = bike[(bike['위도'] > 0) & (bike['경도'] > 0)].copy()
bike_coords = np.vstack([bike_clean['위도'], bike_clean['경도']]).T
bike_tree = cKDTree(bike_coords)
```
> 결측치 제거




```python
# KDTree를 위한 기준 좌표 준비
apt_coords = np.vstack([concat_select['좌표Y'], concat_select['좌표X']]).T
```

```python
# 지하철 KDTree
subway_coords = np.vstack([subway['위도'], subway['경도']]).T
subway_tree = cKDTree(subway_coords)
```

```python
# 따릉이 KDTree (0.0 또는 NaN 제거)
bike_clean = bike[(bike['위도'] > 0) & (bike['경도'] > 0)]
bike_coords = np.vstack([bike_clean['위도'], bike_clean['경도']]).T
bike_tree = cKDTree(bike_coords)
```

```python
# 최단거리
concat_select['지하철_최단거리'] = subway_tree.query(apt_coords)[0]
concat_select['따릉이_최단거리'] = bike_tree.query(apt_coords)[0]
```

```python
# 반경 내 지하철역 개수 (500m ≈ 0.005도)
subway_counts = subway_tree.query_ball_point(apt_coords, r=0.005)
concat_select['지하철_500m내_개수'] = [len(x) for x in subway_counts]
```

```python
# 반경 내 따릉이 대여소 개수 (300m ≈ 0.003도)
bike_counts = bike_tree.query_ball_point(apt_coords, r=0.003)
concat_select['따릉이_300m내_개수'] = [len(x) for x in bike_counts]

concat_select[['지하철_최단거리', '지하철_500m내_개수', '따릉이_최단거리', '따릉이_300m내_개수']].describe()
```

```python
# 시각화 대상 변수 목록
cols = ['지하철_최단거리', '지하철_500m내_개수', '따릉이_최단거리', '따릉이_300m내_개수']
```
> 지하철/따릉이와 거리 기반 변수 생성




```python
# 시각화
plt.figure(figsize=(16, 8))
for i, col in enumerate(cols):
    plt.subplot(2, 2, i + 1)
    sns.histplot(concat_select[col], bins=50, kde=True, color='skyblue')
    plt.title(f'{col} 분포')
    plt.grid(True)
plt.tight_layout()
plt.show()
```

```python
## 분포가 오른쪽 긴꼬리 양상을 보이기 때문에 로그 변환 실행
```

```python
# 로그 변환 적용
concat_select['지하철_최단거리_log'] = np.log1p(concat_select['지하철_최단거리'])
concat_select['따릉이_최단거리_log'] = np.log1p(concat_select['따릉이_최단거리'])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(concat_select['지하철_최단거리_log'], bins=50, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('지하철_최단거리 (로그변환 후)')
axes[0].set_xlabel('log(지하철_최단거리 + 1)')

sns.histplot(concat_select['따릉이_최단거리_log'], bins=50, kde=True, ax=axes[1], color='lightcoral')
axes[1].set_title('따릉이_최단거리 (로그변환 후)')
axes[1].set_xlabel('log(따릉이_최단거리 + 1)')

plt.tight_layout()
plt.show()
```
> 로그 스케일로 분포 정규화

